In [107]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import json
import re
import time
import os
import re

domain = 'https://www.8book.com'

In [108]:
# get_book_data(url)
# Inputs: book's url <str>
# Ouputs: chapters list <list>, book's name <str>, book's type <str>, book's chapters' count <int>

def get_book_data(url):
    chapter_list = []
    name = ''
    type = ''
    count = 0

    html = requests.get(url)
    html.encoding = 'utf8'
    soup = BeautifulSoup(html.text, 'html.parser')

    for i in soup.find_all('meta'):
        if i.get('name') == 'cat': 
            type = i.get('content')
        elif i.get('name') == 'name': 
            name = i.get('content')
        elif i.get('name') == 'count':
            count = int(re.sub(r'\D','',i.get('content')))

    chapters = soup.find_all(id='chapters')[0]
    for i in chapters.findAll('a'):
        chapter_list.append(domain+i.get('href'))
    
    return chapter_list, name, type, count

# get_book_data('https://www.8book.com/novelbooks/104934')[0][1]

In [109]:
# get_article(url_)
# Inputs: chapter's url <str>
# Ouputs: article text <list>

def get_article(url_):
    # url_ = 'https://www.8book.com/read/104934/?20740_2'
    url = f'http://localhost:8050/render.html?url={url_}&timeout=10&wait=0.5'
    html = requests.get(url)
    html.encoding = 'utf8'
    soup = BeautifulSoup(html.text, 'html.parser')

    texts = ''
    for i in soup.find_all(name='div',attrs={'class':'text'}):
        texts += i.text
    texts = texts.replace('\u3000','').replace('\n','')
    texts = re.sub('[a-zA-Z0-9]','',texts)
    return texts

# get_article('https://www.8book.com/read/104934/?20740_2')

In [110]:
book_list = []

a = 1 
while True:

    url = f'https://www.8book.com/booklists/11/{a}.html'
    html = requests.get(url)
    html.encoding = 'utf8'

    try:
        html = html.text.split('<!-- all list -->')[1]
        soup = BeautifulSoup(html, 'html.parser')

        for i in list(set([i.get('href') for i in soup.find_all('a') if i.get('class') != None])):
            book_list.append(domain+i)

        a += 1
        time.sleep(0.2)
    except:
        break

In [111]:
# utils

path = 'atricles/literature'
if not os.path.isdir(path):
    os.makedirs(path)

for book in book_list:
    # get book's info
    id = book.split('/')[-1]
    url = 'https://www.8book.com/novelbooks/'+id
    cp_list, book_name, book_type, book_count = get_book_data(url) 
    info = {'name':book_name,'type':book_type,'count':book_count}

    # if book not exsists, creat file and update the info.
    if  os.path.isdir(path+'/'+id):
        pass 
    else:
        json_ = {'info':info,'chapters':{}}
        with open(path+'/'+id+'.json', "w") as f:
            json.dump(json_, f)

    # check if chapter is null, if it's null, update it.
    with open(path+'/'+id+'.json', "r") as f:
        json_ = json.load(f)
    charpter = json_['chapters']
    for i in tqdm(range(1,len(cp_list)+1)):
        if str(i) in list(charpter.keys()):
            pass
        else:
            try:
                charpter[i] = get_article(cp_list[i-1])
            except:
                pass
        json_['chapters'] = charpter
    
    # save to the file
    with open(path+'/'+id+'.json', "w") as f:
        json.dump(json_, f)
    
    break


 13%|█▎        | 16/120 [00:29<02:45,  1.59s/it]

In [ ]:
# check state
with open(path+'/'+id+'.json', "r") as f:
    json_ = json.load(f)
print((json_))

{'info': {'name': '紅樓夢', 'type': '歷史名著', 'count': 120}, 'chapters': {'1': "通靈說此《石頭記》一書也，故曰甄士隱云云。但書中所記何事何人自己又云：今風塵碌碌，一事無成，忽念及當日所有之女子：一一細考較去，覺其行止見識皆出我之上。我堂堂須眉誠不若彼裙釵，我實愧則有餘，悔又無益，大無可如何之日也。當此日，欲將已往所賴天恩祖德，錦衣紈之時，飫甘饜肥之日，背父兄教育之恩，負師友規訓之德，以致今日一技無成、半生潦倒之罪，編述一集，以告天下；知我之負罪固多，然閨閣中歷歷有人，萬不可因我之不肖，自護己短，一並使其泯滅也。所以蓬牖茅椽，繩床瓦灶，並不足妨我襟懷；況那晨風夕月，階柳庭花，更覺得潤人筆墨。我雖不學無文，又何妨用假語村言敷演出來亦可使閨閣昭傳。復可破一時之悶，醒同人之目，不亦宜乎？故曰賈雨村云云。更于篇中間用夢幻等字，卻是此書本旨，兼寓提醒閱者之意。看官你道此書從何而起說來雖近荒唐，細玩頗有趣味。卻說那女媧氏煉石補天之時，于大荒山無稽崖煉成高十二丈、見方二十四丈大的頑石三萬六千五百零一塊。那媧皇只用了三萬六千五百塊，單單剩下一塊未用，棄在青埂峰下。誰知此石自經鍛煉之後，靈性已通，自去自來，可大可小。因見眾石俱得補天，獨自己無才不得入選，遂自怨自愧，日夜悲哀。一日正當嗟悼之際，俄見一僧一道遠遠而來，生得骨格不凡，豐神迥異，來到這青埂峰下，席地坐談。見著這塊鮮瑩明潔的石頭，且又縮成扇墜一般，甚屬可愛。那僧托于掌上，笑道：形體倒也是個靈物了，只是沒有實在的好處。須得再鐫上幾個字，使人人見了便知你是件奇物，然後攜你到那昌明隆盛之邦、詩禮簪纓之族、花柳繁華地、溫柔富貴鄉那里去走一遭。石頭聽了大喜，因問：不知可鐫何字攜到何方望乞明示。那僧笑道：你且莫問，日後自然明白。說畢，便袖了，同那道人飄然而去，竟不知投向何方。又不知過了幾世幾劫，因有個空空道人訪道求仙，從這大荒山無稽崖青埂峰下經過。忽見一塊大石，上面字跡分明，編述歷歷。空空道人乃從頭一看，原來是無才補天、幻形入世，被那茫茫大士、渺渺真人攜入紅塵、引登彼岸的一塊頑石；上面敘著墮落之鄉、投胎之處，以及家庭瑣事、閨閣閑情、詩詞謎語，倒還全備。只是朝代年紀，失落無考。後面又有一偈云：無才可去補蒼天，枉入紅塵若許年。此系身前身後事，倩誰記去作奇傳空空道人看了一回，曉